### Izbacivanje izbornih predmeta iz realizacije

In [1]:
from model_parser import *
from realizacija_linked_model import *
from collections import defaultdict

In [2]:
raspored = RasporedTermin.read_list_from_file('4_raspored_spojen')
raspored_izmene = RasporedTermin.read_list_from_file('4_raspored_spojen')

schedule = MeetingSchedule.read_entity_from_file('4_svi_fajlovi_spojeni')
schedule_izmene = MeetingSchedule.read_entity_from_file('4_svi_fajlovi_spojeni')

realizacija = Realizacija.read_entity_from_file('5_realizacija')
realizacija_izmene = Realizacija.read_entity_from_file('5_realizacija')

In [3]:
def find_elective_courses(
        raspored: list[RasporedTermin]
) -> set[str]:
    electives = set()
    for termin in raspored:
        grupa = termin.studGrupa.lower()
        if 'prij' in grupa or 'opred' in grupa:
            electives.add(termin.predmet_id)
    return electives

In [4]:
def store_elective_courses(
        electives: list[str],
        predmeti: list[Predmet],
        stud_programi: list[StudijskiProgram],
        file_name: str = 'izborni_predmeti',
        dir_path: str = '../out_data/'
):
    elective_courses = defaultdict(lambda: [])
    for elective_id in electives:
        predmet = next(x for x in predmeti if x.id == elective_id)
        stud_program = next(x for x in stud_programi if x.id == predmet.studijskiProgram)
        elective_courses[stud_program.naziv].append('    ' + predmet.oznaka + ', ' + predmet.naziv)
    
    with open(dir_path + file_name + '.txt', 'w', encoding='utf-8') as out_file:
        for stud_program_name, elective_names in elective_courses.items():
            out_file.write(stud_program_name)
            out_file.write('\n')
            for elective in elective_names:
                out_file.write(elective)
                out_file.write('\n')
            out_file.write('\n')

In [5]:
def remove_elective_courses(
        electives: list[str],
        realizacija: Realizacija
) -> Realizacija:
    for stud_program in realizacija.studijskiProgramPredmeti:
        without_electives = [x for x in stud_program.predmetPredavaci if x.predmetId not in electives]
        stud_program.predmetPredavaci = without_electives
    return realizacija

In [6]:
electives = find_elective_courses(raspored_izmene)
store_elective_courses(electives, schedule_izmene.predmetList, schedule_izmene.studProgramList)
realizacija_izmene = remove_elective_courses(electives, realizacija_izmene)

### Izbacivanje podsmer izbornih predmeta iz realizacije

In [7]:
def find_podsmer_elective_courses(
        predmeti: list[Predmet]
) -> list[str]:
    # dobavi id-eve svih podmser izbornih predmeta
    izborni_podsmer_predmeti_ids = set()

    predmet = next(x for x in predmeti if x.oznaka == 'A332' and x.naziv == 'Maketarstvo')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'A315' and x.naziv == 'Principi i proces umetničkog stvaranja')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'A01ASP' and x.naziv == 'Studio projekat 01A')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'A01ESP' and x.naziv == 'Studio projekat 01E')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'A377' and x.naziv == 'Arhitektonske reprezentacije 3')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'A354' and x.naziv == 'Posebni programi stanovanja')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'F211I1' and x.naziv == 'Projektovanje grafičkih proizvoda')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'F311I2' and x.naziv == 'Virtuelna i proširena stvarnost')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'SE239A' and x.naziv == 'Veb programiranje')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'P1406' and x.naziv == 'Teorija obradnih procesa')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'P3401' and x.naziv == 'Svojstva i primena plastičnih materijala')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'S002A' and x.naziv == 'Ekonomija')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'E251' and x.naziv == 'Sociološki aspekti tehničkog razvoja')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'IAM004' and x.naziv == 'Geometrija diskretnih prostora')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    predmet = next(x for x in predmeti if x.oznaka == 'IAM006' and x.naziv == 'Snimanje i analiza kretanja')
    izborni_podsmer_predmeti_ids.add(predmet.id)

    return izborni_podsmer_predmeti_ids

In [8]:
podsmer_electives = find_podsmer_elective_courses(schedule_izmene.predmetList)
realizacija_izmene = remove_elective_courses(podsmer_electives, realizacija_izmene)

### Zapisivanje izmena

In [9]:
ReadWrite.write_to_file(realizacija_izmene, '6_realizacija_bez_izbornih')